<a href="https://colab.research.google.com/github/EikEUFS/BuscaOtimizacao/blob/master/BERT_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etapa 1: Instalação e importação das bibliotecas

- SQUAD Link: https://rajpurkar.github.io/SQuAD-explorer/

In [ ]:
!pip install sentencepiece
!pip install tf-models-nightly
!pip install tf-nightly

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import tensorflow_hub as hub
from official.nlp.bert.tokenization import FullTokenizer
from official.nlp.bert.input_pipeline import create_squad_dataset
from official.nlp.data.squad_lib import generate_tf_record_from_json_file
from official.nlp import optimization
from official.nlp.data.squad_lib import read_squad_examples
from official.nlp.data.squad_lib import FeatureWriter
from official.nlp.data.squad_lib import convert_examples_to_features
from official.nlp.data.squad_lib import write_predictions 

In [ ]:
import numpy as np
import math
import random
import time
import json
import collections
import os
from google.colab import drive

# Etapa 2: Pré-processamento

In [ ]:
drive.mount("/content/drive")

In [ ]:
input_meta_data = generate_tf_record_from_json_file(
    "/content/drive/MyDrive/PLN/BERT-Q&A/train-v1.1.json",
    "/content/drive/MyDrive/PLN/BERT-Q&A/vocab.txt",
    "/content/drive/MyDrive/PLN/BERT-Q&A/train-v1.1.tf_record"
)

In [ ]:
with tf.io.gfile.GFile("/content/drive/MyDrive/PLN/BERT-Q&A/train_meta_data", "w") as writer:
  writer.write(json.dumps(input_meta_data, indent=4) + "\n")

In [ ]:
BATCH_SIZE = 4
train_dataset = create_squad_dataset(
    "/content/drive/MyDrive/PLN/BERT-Q&A/train-v1.1.tf_record",
    input_meta_data['max_seq_length'], # 384
    BATCH_SIZE,
    is_training = True
)

# Etapa 3: Construção do modelo

## Camada Squad

In [ ]:
class BertSquardLayer(tf.keras.layers.Layer):
  def __init__(self):
    super(BertSquardLayer, self).__init__()
    self.final_dense = tf.keras.layers.Dense(units=2,
                                             kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))
    
  def call(self, inputs):
    logits = self.final_dense(inputs) # (batch_size, seq_len, 2)
    logits = tf.transpose(logits, [2, 0, 1]) # (2, batch_size, seq_len)
    unstacked_logits = tf.unstack(logits, axis=0) # [(batch_size, seq_len), (batch_size, seq_len)]
    return unstacked_logits[0], unstacked_logits[1] 

## Modelo completo

In [ ]:
class BERTSquad(tf.keras.Model):
  def __init__(self, name="bert_squad"):
    super(BERTSquad, self).__init__(name=name)
    self.bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                                     trainable = True)
    self.squad_layer = BertSquardLayer()

  def apply_bert(self, inputs):
    _, sequence_output = self.bert_layer([inputs["input_word_ids"],
                                          inputs["input_mask"],
                                          inputs["input_type_ids"]])
    return sequence_output

  def call(self, inputs):
    seq_outputs = self.apply_bert(inputs)
    start_logits, end_logits = self.squad_layer(seq_outputs)
    return start_logits, end_logits

# Etapa 4: Treinamento

## Criação da IA

In [ ]:
TRAIN_DATA_SIZE = 88641
NB_BATCHES_TRAIN = 2000
BATCH_SIZE = 4
NB_EPOCHS = 3
INIT_LR = 5e-5
WARMUP_STEPS = int(NB_BATCHES_TRAIN * 0.1)

In [ ]:
train_dataset_light = train_dataset.take(NB_BATCHES_TRAIN)

In [ ]:
bert_squad = BERTSquad()

In [ ]:
optimizer = optimization.create_optimizer(init_lr=INIT_LR,
                                          num_train_steps=NB_BATCHES_TRAIN,
                                          num_warmup_steps = WARMUP_STEPS)

In [ ]:
def squad_loss_fn(labels, model_outputs):
  start_positions = labels['start_positions']
  end_positions = labels['end_positions']
  start_logits, end_logits = model_outputs

  start_loss = tf.keras.backend.sparse_categorical_crossentropy(start_positions, start_logits, from_logits=True)
  end_loss = tf.keras.backend.sparse_categorical_crossentropy(end_positions, end_logits, from_logits=True)

  total_loss = (tf.reduce_mean(start_loss) + tf.reduce_mean(end_loss)) / 2

  return total_loss

In [ ]:
train_loss = tf.keras.metrics.Mean(name="train_loss")

In [ ]:
bert_squad.compile(optimizer, squad_loss_fn)

In [ ]:
checkpoint_path = "/content/drive/MyDrive/PLN/BERT-Q&A/"
ckpt = tf.train.Checkpoint(bert_squad=bert_squad)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

## Treinamento personalizado

In [ ]:
for epoch in range(NB_EPOCHS):
  print("Start of epoch {}".format(epoch + 1))
  start = time.time()

  train_loss.reset_states()

  for (batch, (inputs, targets)) in enumerate(train_dataset_light):
    with tf.GradientTape() as tape:
      model_outputs = bert_squad(inputs)
      loss = squad_loss_fn(targets, model_outputs)
    
    gradients = tape.gradient(loss, bert_squad.trainable_variables)
    optimizer.apply_gradients(zip(gradients, bert_squad.trainable_variables))

    train_loss(loss)

    if batch % 50 == 0:
      print("Epoch {} Batch {} Loss {:.4f}".format(epoch+1, batch, train_loss.result()))

    if batch % 500 == 0:
      ckpt_save_path = ckpt_manager.save()
      print("Saving checkpoint for epoch {} at {}".format(epoch+1, ckpt_save_path))
  
  print("Time taken for 1 epoch: {} secs\n".format(time.time() - start))

# Etapa 5: Avaliação

## Preparação da avaliação

Obter o conjunto dev na sessão

In [ ]:
eval_examples = read_squad_examples(
    "/content/drive/MyDrive/PLN/BERT-Q&A/dev-v1.1.json",
    is_training=False,
    version_2_with_negative=False)

Definição da função que gerará o arquivo tf_record

In [ ]:
eval_writer = FeatureWriter(
    filename=os.path.join("/content/drive/MyDrive/PLN/BERT-Q&A/",
                          "eval.tf_record"),
    is_training=False)

Criação do tokenizador

In [ ]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)
vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

Criação da função que adicona características na lista eva_features

In [ ]:
def _append_feature(feature, is_padding):
    if not is_padding:
        eval_features.append(feature)
    eval_writer.process_feature(feature)

Criação das características e geração do arquivo tf.record

In [ ]:
eval_features = []
dataset_size = convert_examples_to_features(
    examples=eval_examples,
    tokenizer=tokenizer,
    max_seq_length=384,
    doc_stride=128,
    max_query_length=64,
    is_training=False,
    output_fn=_append_feature,
    batch_size=4)

In [ ]:
eval_writer.close()

Carregamento da base de dados

In [ ]:
BATCH_SIZE = 4

eval_dataset = create_squad_dataset(
    "/content/drive/MyDrive/PLN/BERT-Q&A/eval.tf_record",
    384, # input_meta_data['max_seq_length'],
    BATCH_SIZE,
    is_training=False)

## Fazendo as previsões com as funções do Google

In [ ]:
RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])

In [ ]:
def get_raw_results(predictions):
  for unique_ids, start_logits, end_logits in zip(predictions['unique_ids'],
                                                  predictions['start_logits'],
                                                  predictions['end_logits']):
    yield RawResult(
        unique_id=unique_ids.numpy(),
        start_logits=start_logits.numpy().tolist(),
        end_logits=end_logits.numpy().tolist()
    )

In [ ]:
all_results = []
for count, inputs in enumerate(eval_dataset):
  x, _ = inputs
  #print(x)
  unique_ids = x.pop("unique_ids")
  start_logits, end_logits = bert_squad(x, training=False)
  output_dict = dict(
      unique_ids = unique_ids,
      start_logits=start_logits,
      end_logits=end_logits
  )
  for result in get_raw_results(output_dict):
    all_results.append(result)
  
  print(count)
  if count % 100 == 0:
    print("{}/{}".format(count, 2709))

In [ ]:
len(all_results)

In [ ]:
all_results[0]

In [ ]:
output_prediction_file = "/content/drive/MyDrive/PLN/BERT-Q&A/predictions.json"
output_nbest_file = "/content/drive/MyDrive/PLN/BERT-Q&A/nbest_predictions.json"
output_null_log_odds_file = "/content/drive/MyDrive/PLN/BERT-Q&A/null_odds.json"

In [ ]:
write_predictions(
    eval_examples,
    eval_features,
    all_results,
    20,
    30,
    True,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    verbose=False
)

## Fazendo previsões personalizadas

### Criação do dicionário de inputs

In [ ]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)
vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def is_whitespace(c):
  if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
    return True
  return False

In [ ]:
is_whitespace("a")

False

In [ ]:
is_whitespace("\n")

True

In [ ]:
is_whitespace(" ")

True

In [ ]:
def whitespace_split(text):
  doc_tokens = []
  prev_is_whitespace = True
  for c in text:
    if is_whitespace(c):
      prev_is_whitespace = True
    else:
      if prev_is_whitespace:
        doc_tokens.append(c)
      else:
        doc_tokens[-1] += c
      prev_is_whitespace = False
  return doc_tokens

In [ ]:
whitespace_split("My dog likes strawberries.")

['My', 'dog', 'likes', 'strawberries.']

In [ ]:
tokenizer.tokenize("My")

['my']

In [ ]:
t = tokenizer.tokenize("strawberries")
t

['straw', '##berries']

In [ ]:
len(t)

2

In [ ]:
[1] * len(t)

[1, 1]

In [ ]:
def tokenize_context(text_words):
  text_tok = []
  tok_to_word_id = []
  for word_id, word in enumerate(text_words):
    word_tok = tokenizer.tokenize(word)
    text_tok += word_tok
    tok_to_word_id += [word_id] * len(word_tok) 
  return text_tok, tok_to_word_id

In [ ]:
tokenize_context(whitespace_split("My dog likes strawberries."))

(['my', 'dog', 'likes', 'straw', '##berries', '.'], [0, 1, 2, 3, 3, 3])

In [ ]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id
    return seg_ids

In [ ]:
my_context = '''Neoclassical economics views inequalities in the distribution of income as arising from differences in value added by labor, capital and land. Within labor income distribution is due to differences in value added by different classifications of workers. In this perspective, wages and profits are determined by the marginal value added of each economic actor (worker, capitalist/business owner, landlord). Thus, in a market economy, inequality is a reflection of the productivity gap between highly-paid professions and lower-paid professions.'''

Neoclassical economics views inequalities in the distribution of income as arising from differences in value added by labor, capital and land. Within labor income distribution is due to differences in value added by different classifications of workers. In this perspective, wages and profits are determined by the marginal value added of each economic actor (worker, capitalist/business owner, landlord). Thus, in a market economy, inequality is a reflection of the productivity gap between highly-paid professions and lower-paid professions.

In [ ]:
my_question = '''What are examples of economic actors?'''
#my_question = '''What philosophy of thought addresses wealth inequality?'''
#my_question = '''In a market economy, what is inequality a reflection of?'''

In [ ]:
def create_input_dict(question, context):
  question_tok = tokenizer.tokenize(my_question)
  #print(question_tok)

  context_words = whitespace_split(context)
  #print(context_words)
  context_tok, context_tok_to_word_id = tokenize_context(context_words)
  #print(context_tok)
  #print(context_tok_to_word_id)

  input_tok = question_tok + ["[SEP]"] + context_tok + ["[SEP]"]
  #print(input_tok)
  input_tok += ["[PAD]"]*(384-len(input_tok))
  #print(input_tok)

  input_dict = {}
  input_dict["input_word_ids"] = tf.expand_dims(tf.cast(get_ids(input_tok), tf.int32), 0)
  input_dict["input_mask"] = tf.expand_dims(tf.cast(get_mask(input_tok), tf.int32), 0)
  input_dict["input_type_ids"] = tf.expand_dims(tf.cast(get_segments(input_tok), tf.int32), 0)

  return input_dict, context_words, context_tok_to_word_id, len(question_tok)

In [ ]:
my_input_dict, my_context_words, context_tok_to_word_id, question_tok_len = create_input_dict(my_question, my_context)

In [ ]:
my_input_dict["input_word_ids"]

<tf.Tensor: shape=(1, 384), dtype=int32, numpy=
array([[ 2054,  2024,  4973,  1997,  3171,  5889,  1029,   102, 22657,
         5543,  5328,  1999,  2063, 26426,  6447,  1999,  1996,  4353,
         1997,  3318,  2004, 17707,  2013,  5966,  1999,  3643,  2794,
         2011,  4450,  1010,  3007,  1998,  2455,  1012,  2306,  4450,
         3318,  4353,  2003,  2349,  2000,  5966,  1999,  3643,  2794,
         2011,  2367, 26739,  1997,  3667,  1012,  1999,  2023,  7339,
         1010, 12678,  1998, 11372,  2024,  4340,  2011,  1996, 14785,
         3643,  2794,  1997,  2169,  3171,  3364,  1006,  7309,  1010,
        19640,  1013,  2449,  3954,  1010, 18196,  1007,  1012,  2947,
         1010,  1999,  1037,  3006,  4610,  1010, 16440,  2003,  1037,
         9185,  1997,  1996, 15836,  6578,  2090,  3811,  1011,  3825,
        22797,  1998,  2896,  1011,  3825, 22797,  1012,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [ ]:
my_input_dict["input_mask"]

<tf.Tensor: shape=(1, 384), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
my_input_dict["input_type_ids"]

<tf.Tensor: shape=(1, 384), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
print(my_context_words)

['Neoclassical', 'economics', 'views', 'inequalities', 'in', 'the', 'distribution', 'of', 'income', 'as', 'arising', 'from', 'differences', 'in', 'value', 'added', 'by', 'labor,', 'capital', 'and', 'land.', 'Within', 'labor', 'income', 'distribution', 'is', 'due', 'to', 'differences', 'in', 'value', 'added', 'by', 'different', 'classifications', 'of', 'workers.', 'In', 'this', 'perspective,', 'wages', 'and', 'profits', 'are', 'determined', 'by', 'the', 'marginal', 'value', 'added', 'of', 'each', 'economic', 'actor', '(worker,', 'capitalist/business', 'owner,', 'landlord).', 'Thus,', 'in', 'a', 'market', 'economy,', 'inequality', 'is', 'a', 'reflection', 'of', 'the', 'productivity', 'gap', 'between', 'highly-paid', 'professions', 'and', 'lower-paid', 'professions.']


In [ ]:
print(context_tok_to_word_id)

[0, 1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 17, 18, 19, 20, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 36, 37, 38, 39, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 54, 54, 55, 55, 55, 56, 56, 57, 57, 57, 58, 58, 59, 60, 61, 62, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 72, 72, 73, 74, 75, 75, 75, 76, 76]


In [ ]:
question_tok_len

7

### Previsões

In [ ]:
start_logits, end_logits = bert_squad(my_input_dict, training = False)

In [ ]:
start_logits[0]

<tf.Tensor: shape=(384,), dtype=float32, numpy=
array([-6.1141114, -5.94246  , -6.3286915, -6.2150784, -5.1629057,
       -6.5666614, -6.524823 , -6.047181 , -3.0152621, -5.442794 ,
       -5.2678294, -3.7595422, -5.82138  , -5.8799143, -6.3127155,
       -5.7444506, -5.602176 , -5.420313 , -6.1502676, -5.844705 ,
       -5.815895 , -5.468899 , -5.5814114, -3.8503177, -6.3701677,
       -5.819021 , -6.513452 , -6.0749454, -3.1359966, -6.352033 ,
       -5.82494  , -6.5621686, -6.1847935, -5.685253 , -3.5545986,
       -2.259898 , -5.3562   , -6.234395 , -6.279558 , -5.474968 ,
       -5.883649 , -3.6245663, -6.2983747, -5.22129  , -6.5412545,
       -5.622674 , -3.0834582, -4.1872306, -6.0717278, -4.2317224,
       -5.4931755, -5.1407843, -5.433543 , -5.426195 , -4.3503265,
       -1.2661135, -5.970591 , -5.678217 , -5.953099 , -5.90874  ,
       -5.8279862, -5.117585 , -1.792199 , -5.690749 , -6.270339 ,
       -6.1369176, -3.3721244, -3.6687026, -4.781014 , -2.3619018,
        3.1457

In [ ]:
question_tok_len

7

In [ ]:
start_logits_context = start_logits.numpy()[0, question_tok_len + 1:]

In [ ]:
start_logits_context

array([-3.0152621, -5.442794 , -5.2678294, -3.7595422, -5.82138  ,
       -5.8799143, -6.3127155, -5.7444506, -5.602176 , -5.420313 ,
       -6.1502676, -5.844705 , -5.815895 , -5.468899 , -5.5814114,
       -3.8503177, -6.3701677, -5.819021 , -6.513452 , -6.0749454,
       -3.1359966, -6.352033 , -5.82494  , -6.5621686, -6.1847935,
       -5.685253 , -3.5545986, -2.259898 , -5.3562   , -6.234395 ,
       -6.279558 , -5.474968 , -5.883649 , -3.6245663, -6.2983747,
       -5.22129  , -6.5412545, -5.622674 , -3.0834582, -4.1872306,
       -6.0717278, -4.2317224, -5.4931755, -5.1407843, -5.433543 ,
       -5.426195 , -4.3503265, -1.2661135, -5.970591 , -5.678217 ,
       -5.953099 , -5.90874  , -5.8279862, -5.117585 , -1.792199 ,
       -5.690749 , -6.270339 , -6.1369176, -3.3721244, -3.6687026,
       -4.781014 , -2.3619018,  3.1457474, -5.037569 , -3.4424314,
       -5.3498964, -5.504025 , -4.6517553, -5.51655  , -3.3634362,
       -3.4924552, -5.4695644, -5.56817  , -6.1929235, -4.5492

In [ ]:
end_logits_context = end_logits.numpy()[0, question_tok_len + 1:]

In [ ]:
print(context_tok_to_word_id)

[0, 1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 17, 18, 19, 20, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 36, 37, 38, 39, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 54, 54, 55, 55, 55, 56, 56, 57, 57, 57, 58, 58, 59, 60, 61, 62, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 72, 72, 73, 74, 75, 75, 75, 76, 76]


In [ ]:
start_word_id = context_tok_to_word_id[np.argmax(start_logits_context)]
start_word_id

54

In [ ]:
end_word_id = context_tok_to_word_id[np.argmax(end_logits_context)]
end_word_id

57

### Resposta final

In [ ]:
predicted_answer = ' '.join(my_context_words[start_word_id:end_word_id + 1])
print("The answer to:\n" + my_question + "\nis:\n" + predicted_answer)

The answer to:
What are examples of economic actors?
is:
(worker, capitalist/business owner, landlord).


In [ ]:
from IPython.core.display import HTML
display(HTML(f'<h2>{my_question.upper()}</h2>'))
marked_text = str(my_context.replace(predicted_answer, f"<mark>{predicted_answer}</mark>"))
#print(marked_text)
display(HTML(f"""<blockquote> {marked_text} </blockquote>"""))